In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install timm
import torch; torch._dynamo.config.recompile_limit = 64;

In [2]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.6: Fast Gemma3N patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.339 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [11]:
from transformers import TextStreamer
# Helper function for inference
def do_gemma_3n_inference(messages, max_new_tokens = 128):
    # _ = model.generate(
    #     **tokenizer.apply_chat_template(
    #         messages,
    #         add_generation_prompt = True, # Must add for generation
    #         tokenize = True,
    #         return_dict = True,
    #         return_tensors = "pt",
    #     ).to("cuda"),
    #     max_new_tokens = max_new_tokens,
    #     temperature = 1.0, top_p = 0.95, top_k = 64,
    #     streamer = TextStreamer(tokenizer, skip_prompt = True),
    #)
    # 1. Prepare inputs
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")

    # 2. Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 0.1, # Lower temperature is better for navigation (more factual)
        do_sample = True,
        # Keep the streamer so you can still watch it work in the notebook
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

    # 3. Decode only the NEW tokens (the answer)
    input_length = inputs.input_ids.shape[1]
    new_tokens = outputs[0][input_length:]
    response_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    return response_text

In [5]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 205.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 120.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.24.0 requires transformers>=4.56.1, but you have transformers 4.55.4 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
import os
import cv2
import csv
from PIL import Image
from pathlib import Path

def extract_frames_from_path(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video file.")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate the step size (logic from your first source)
    step = total_frames // num_frames
    frames = []

    for i in range(num_frames):
        frame_idx = i * step
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert BGR (OpenCV) to RGB (PIL) without rotation
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frames.append(img)

    cap.release()
    return frames

In [13]:
video_folder = "videos"  #folder name
output_csv = "gemma3n-4b_inference_results.csv"
results = []

# Get list of all video files
video_files = [f for f in Path(video_folder).iterdir() if f.suffix in ['.mp4', '.mov', '.avi']]

print(f"Found {len(video_files)} videos. Starting processing...")

for video_path in video_files:
    print(f"Processing: {video_path.name}...")
    
    # 1. Extract the "Frame Thing" (10 snapshots)
    video_frames = extract_frames_from_path(video_path, num_frames=10)
    
    if not video_frames:
        continue

    # 2. Build the message
    messages = [
        {
            "role": "user",
            "content": [
                *[{"type": "image", "image": img} for img in video_frames],
                {"type": "text", "text": "You are an assistive navigation system for a visually impaired user. Analyze the provided video from the user's forward perspective. Identify all the immediate, high-risk obstructions. State the obstruction's location using the 12-hour clock face. Process the provided video generate a single, actionable safety alert."}
            ],
        },
    ]

    # 3. Inference
    try:
        response = do_gemma_3n_inference(messages, max_new_tokens=256)
        results.append([video_path.name, response])
    except Exception as e:
        print(f"Error processing {video_path.name}: {e}")

# --- Save to CSV ---
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["video_name", "model_output"])
    writer.writerows(results)

print(f"\nSuccess! Saved results to {output_csv}")

Found 15 videos. Starting processing...
Processing: 20240918-youtube_short_83bc88612ff6a00628af923c9ff461cb_2m9s.mp4...
## Safety Alert:

**Immediate High-Risk Obstruction:** There is a glass railing located at approximately 11:00 on the 12-hour clock face. Please be mindful of this railing as you navigate the walkway. 

**Reasoning:** The railing is a potential tripping hazard if not detected. 

**Additional Notes:** The walkway appears to be busy with people, so maintaining awareness of your surroundings is important. 



<end_of_turn>
Processing: 20240918-youtube_short_b2393a2b8d5fe9bc63504236205ceb98_13s.mp4...
## Safety Alert:

**High-risk obstruction:** A metal railing is located directly in your path at approximately the 11:00 position. Please be cautious as you approach this area. 

**Additional observations:** There are people seated at tables outside several establishments along the street, and some parked bicycles. Be mindful of these individuals and objects as you navigate 